In [ ]:
import siibra
from nilearn import plotting

# Assignment of an MNI coordinate to Julich-Brain probabilistic cytoarchitectonic maps
`siibra` can use the probabilistic maps of the Julich-Brain cytorachitectonic atlas to make a probabilistic assignment of a coordinate with error radius (modelled as a Gaussian) to brain regions.

In [ ]:
# some coordinates  in MNI space, e.g. from an EEG contact
xyz_mni = [
    (31.0, -89.6, -6.475),  # a real sEEG contact point
    (27.75, -32.0, 63.725)    # this should be in PostCG, right hemisphere
          ]

# a confidence radius for the coordinate, i.e. the expected localization error.
radius_mm = 5

We instantiate the human atlas with the Julich-brain probabilistic cytoarchitectonic maps, and then ask it to assign regions. It will return a sorted list of probabilities with the corresponding regions. 

In [ ]:
atlas = siibra.atlases.MULTILEVEL_HUMAN_ATLAS
atlas.select(parcellation="julich")
assignments = atlas.assign_coordinates("mni152",xyz_mni,radius_mm)

We look at the best fitting maps at the first requested location.

In [ ]:
for region,pmap,scores in assignments[0]:
    if scores['contains']>.5: 
        plotting.plot_stat_map(pmap, 
                               cut_coords=xyz_mni[0],
                               title=f"{region.name} ({scores['contains']*100:.1f}%)")

# Connectivity profiles of the most probable brain region

`siibra`'s key feature is access to regional data features, including connectivity profiles from different projects. This can be used to check the connection strengh of the most likely region assignment to the MNI coordinate.

In [ ]:
# get profiles for the top assigned region 
closest_region,pmap,scores = next(iter(assignments[0]))
atlas.select_region(closest_region)
profiles = atlas.get_features(
    siibra.modalities.ConnectivityProfile)

We will create plots of the connection strength to the 20 most strongly connected regions, for each of the returned profiles. Note that the profiles come from different connectivity datasets. The `src_info` and `src_name` attributes tell us more about each dataset.

First, we decode the profiles with the parcellation object. This will convert the column names of the connectivity profile to explicit brain region objects, helping to disambiguiate region names.

In [ ]:
siibra.logger.setLevel("ERROR") # ignore warnings 
decoded_profiles = [p.decode(atlas.selected_parcellation)
                   for p in profiles]
siibra.logger.setLevel("INFO")
p = decoded_profiles[0]
target_regions = [region for strength,region in p[:20]]
target_regions

We build a plotting function for the decoded profiles, which takes the N most strongly connected regions of the first profile, and then plots the connection strengths of all found profiles for those N target regions.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# a function to create a nice plot of multiple profiles from different
# data sources
def plot_connectivity_profiles(profiles,target_regions):
    # Let's plot the so obtained regions and their strenghts
    N = len(target_regions)
    xticks = range(N)
    fig  = plt.figure()
    ax1  = fig.add_subplot(211)
    ax1.set_xticks(xticks)
    ax1.set_xticklabels([r.name for r in target_regions], 
                        rotation=45,fontsize=10,ha='right')
    
    for p in profiles:
        probs = {region:prob for prob,region in p}
        y = [probs[r] if r in probs else 0 
             for r in target_regions ]
        strengths = []
        ax1.plot(xticks,y,'.-',lw=1)

    ax1.grid(True)
    return fig

In [ ]:
fig = plot_connectivity_profiles(decoded_profiles,target_regions)
fig.legend([p.src_name for p in profiles],
           loc='upper left', bbox_to_anchor=(1.05, 1.0),
           prop={'size': 9})
fig.gca().set_title(f"Connection strengths from area {atlas.selected_region}")
plt.show()